# Critical Introduction to Natural Language Processing

Using only the raw text, we'll derive and explore the semantic properties of its words.

## Imports

Python code in one module gains access to the code in another module by the process of importing it. The import statement is the most common way of invoking the import machinery, but it is not the only way.

In [ ]:
from __future__ import absolute_import, division, print_function

First we import common system-tools etc. here that are not directly connected to NLP


In [ ]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

In [2]:
import nltk
import gensim.models.word2vec as w2v
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


You will probably run into an "ModuleNorFoundError" here. THis means that the needed module is not installed on your system.
You can do that in the anaconda command prompt:
for example: <b>"conda install -c anaconda nltk"</b> or <b>"conda install -c anaconda gensim"</b> <br> for detailed information refer to https://docs.anaconda.com/anaconda/user-guide/tasks/install-packages/ <br>


In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Set up logging**

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**Download NLTK tokenizer models (only the first time)**

In [5]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fmx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fmx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Set a Text

In [6]:
sentence = "The oppressed are allowed once every few years to decide which particular representatives of the oppressing class are to represent and repress them."

**Build your vocabulary (word tokenization)**

In [7]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Given a character sequence and a defined document unit, tokenization is the task of chopping it up into pieces, called tokens , perhaps at the same time throwing away certain characters, such as punctuation.

In [8]:
#convert into a list of words
#rtemove unnnecessary,, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [9]:
#sentence where each word is tokenized
words = sentence_to_wordlist(sentence)

In [10]:
print(sentence)
print(words)

The oppressed are allowed once every few years to decide which particular representatives of the oppressing class are to represent and repress them.
['The', 'oppressed', 'are', 'allowed', 'once', 'every', 'few', 'years', 'to', 'decide', 'which', 'particular', 'representatives', 'of', 'the', 'oppressing', 'class', 'are', 'to', 'represent', 'and', 'repress', 'them']


In [11]:
num_tokens = len(words)
print("The corpus contains {0:,} tokens".format(num_tokens))

The corpus contains 23 tokens


Now we sort the words lexically

In [12]:
vocab = sorted(set(words))
vocab_size = len(vocab)
', '.join(vocab)
print("The size of the vocabulary is {0:,} words".format(vocab_size))

The size of the vocabulary is 21 words


## Create a one-hot vector representation

In [13]:
onehot_vectors = np.zeros((num_tokens, vocab_size), int)
print(onehot_vectors)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 

In [14]:
for i, word in enumerate(words):
    onehot_vectors[i, vocab.index(word)] = 1
' '.join(vocab)

'The allowed and are class decide every few of once oppressed oppressing particular represent representatives repress the them to which years'

In [15]:
print(onehot_vectors)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 

In [24]:
pd.DataFrame(onehot_vectors, words, columns=vocab)

,The,allowed,and,are,class,decide,every,few,of,once,...,oppressing,particular,represent,representatives,repress,the,them,to,which,years
The,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oppressed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
are,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
allowed,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
once,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
every,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
few,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
years,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
to,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
decide,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df = pd.DataFrame(onehot_vectors, columns=vocab)
df[df == 0] = ''
df

,The,allowed,and,are,class,decide,every,few,of,once,...,oppressing,particular,represent,representatives,repress,the,them,to,which,years
0,1,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,1,,,,,,,...,,,,,,,,,,
3,,1,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,1,...,,,,,,,,,,
5,,,,,,,1,,,,...,,,,,,,,,,
6,,,,,,,,1,,,...,,,,,,,,,,
7,,,,,,,,,,,...,,,,,,,,,,1
8,,,,,,,,,,,...,,,,,,,,1,,
9,,,,,,1,,,,,...,,,,,,,,,,
